---
title: "Who is Exposed to Coastal Hazards in Puerto Rico?"
author: 
   - Deborah Balk 
   - Kytt MacManus
   - Hieu Tran 
   - Camilla Green 
   - Shemontee Chowdhury
format: 
    html
bibliography: lecz-references.bib
---

## Datasets:
- VIIRS Nighttime Lights
- LECZ
- NHGIS
- LiDAR SAR

## Areas of Interest (AOIs):
- Puerto Rico (PRI)

## Functions:
- Image segmentation
- Validation with SAR or LiDAR

# Overview

In this lesson, you will use the dataset delineating [Low Elevation Coastal Zones](https://sedac.ciesin.columbia.edu/data/set/lecz-low-elevation-coastal-zones) from the NASA Socioeconomic Data and Applications Center (SEDAC) website along with census data from the [US Census Bureau](https://www.census.gov/data.html). 

You will perform various preprocessing tasks to prepare the raw spatial data for analysis. These steps include exploring the dataset with visualizations and thematic mapping tools. You will then learn how to generate summary statistics based on combinations of these two layers, over two different time periods. 

You will learn how to perform numerous data manipulations, create statistical summaries of population-at-risk (and related housing characteristics), and examine a decade of change throughout this lesson.

# Learning Objectives

- Become familiar with Low Elevation Coastal Zones (LECZs) and explain their significance (as well as limitations) in assessing coastal hazard exposure.
- Access, integrate, explore, and use LECZ data from NASA SEDAC and demographic data from the US Census Bureau for Puerto Rico.
- Assess decadal changes (2010–2020) in population and housing characteristics in coastal versus non-coastal zones in Puerto Rico.
- Create regional and local scale maps and statistical figures of exposure and decadal change.
- Identify venues for sharing output (for example, discussion board associated with data, policy briefs, op-eds).

# Introduction

Low Elevation Coastal Zones (LECZs) have been defined globally, with population estimates for areas below 5m and 10m elevations [@mcgranahan2007; @macmanus2021]. In the continental U.S. (CONUS), 1 in 10 people live in the 10m LECZ, and studies highlight that urban residents, people of color, and older adults are disproportionately exposed. For instance, about 1 in 5 urban Black residents live in this zone [@tagtachian2023; @hauer2020sea].

This is the sneak peak of what LECZ looks like:
![](data/lecz_pr/lecz_satellite.png)

You may wonder why studies of the “entire” US often restrict themselves to the CONUS? The simplest answer is limitations either data or computational power. For example, this happens because of incomplete coverage in one data set or another. Some US territories may not collect the full suite of census variables that are collected in CONUS. For example the detail on housing characteristics is limited in Guam, Northern Mariana Islands, US Virgin Islands and American Samoa, and the Census’ American Community Survey is not conducted in any of the territories, though Puerto Rico conducts its own Community Survey. In some other cases, data collected from satellites (such as SRTM) have variable accuracy toward polar regions [U.S Census Bureau](https://docs.google.com/document/d/1bKYopZoMLCD2djl2vc3pwNSKwGDk6TKRI79swefIvKY/edit?tab=t.0). 

Another possible reason for omission outside of CONUS could be computational challenges or limitations. For instance US territories are subject to different map projections, which implies the need for additional functions in processing algorithms to account for spatial variations and to unify spatial structures.

# Additional Resources

### “What is a Map Projection?”
Map projections have existed for thousands of years. They help map makers render the spherical surface of the Earth flat -- so it can be seen on a piece of paper or computer screen, and so that the unit of measure is uniform throughout the area of interest. As a result, map projections distort something -- area, shape, distance, direction -- to make this possible. 

Here are some resources to learn more about map projections:

[A brief video explainer](https://www.youtube.com/watch?v=wlfLW1j05Dg)

[A brief guide from USGS](https://pubs.usgs.gov/gip/70047422/report.pdf)

There are many resources to guide a new learner, so enjoy learning! 

# Accessing Data

This lesson uses the Python language. 

These are the equired packages to run this lesson `impumspy`, `pandas`, `arcgis`, `zipfile`, and `numpy`. Optional packages include `glob`, `geopandas`, and `earthaccess`.

In [ ]:
# from ipumspy import readers, ddi
# from ipumspy.api import IpumsApiClient
# from ipumspy import AggregateDataExtract


# from dotenv import load_dotenv
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.colors as colors
# import os
# from matplotlib.ticker import FuncFormatter
import arcgis
from arcgis.gis import GIS

# # from arcgis.map import Map
# from arcgis.raster import Raster
# from arcgis.features import GeoAccessor

# from arcgis.map.renderers import (
#     ClassBreaksRenderer,
#     ClassBreakInfo,
#     UniqueValueRenderer,
#     UniqueValueInfo,
#     SizeInfoVisualVariable,
# )

# from arcgis.mapping.symbol import SimpleLineSymbol, SimpleFillSymbol

# import geopandas as gpd
# import glob
# from zipfile import ZipFile
# import earthaccess as ea
# import requests
# import pprint
# import re



# Standard libraries
import os
import glob
import re
from zipfile import ZipFile
from dotenv import load_dotenv
import pprint
import requests

# Data manipulation and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FuncFormatter

# Geospatial libraries
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.raster import Raster
import arcgis.mapping  # For using WebMap, MapView, etc.

# Earthdata access
import earthaccess as ea

# IPUMS API and DDI access
from ipumspy.api import IpumsApiClient
from ipumspy import AggregateDataExtract, readers, ddi

from ipumspy import readers, ddi
from ipumspy.api import IpumsApiClient
from ipumspy import AggregateDataExtract


This lesson uses arcgis version of 2.4.0 or higher:

In [ ]:
# Check the arcgis version for mapping properly 
arcgis.__version__

If arcgis version is lower, use `pip install`:

In [ ]:
pip install arcgis==2.4.1.1

## Using IPUMS API to pull U.S Census Data for Puerto Rico


### Registering to IPUMS and the National Historical Geographic Information System (NHGIS) 

In order to retrieve an IPUMS API Key, you will have to register for an account for IPUMS and request your [API Key](https://account.ipums.org/api_keys).

Additionally register, to [The National Historical Geographic Information System (NHGIS)](https://uma.pop.umn.edu/nhgis/registration)

After you requested your IPUMS API, to the NHGIS, store it in `os.env` format. You will need your registration email and the API Key:


In [ ]:
from dotenv import load_dotenv
import os
from ipumspy.api import IpumsApiClient
import pprint

# Step 1: Load API key from .env file
load_dotenv()
IPUMS_API_KEY = os.getenv("EMAIL", "API_KEY")

if IPUMS_API_KEY is None:
    raise ValueError("API key not found. Make sure IPUMS_API_KEY is defined in your .env file.")

# Step 2: Initialize IPUMS client
ipums = IpumsApiClient(IPUMS_API_KEY)


In [ ]:
ipums 

Getting shapefile metadata in order to get the filename for downloading the shapefile in the below chunk.

This block will be returning the shapefile name of interest so that we can download it in the next block


In [ ]:
# After registering to NHGIS, please run this code

for page in ipums.get_metadata_catalog("nhgis", metadata_type="shapefiles"):
    for shapefile in page["data"]:
        if shapefile["extent"] in ["Puerto Rico", "United States"]:
            if shapefile["geographicLevel"] in ["ZCTA", "ZIP Code Tabulation Area"]:
                print(f"Name: {shapefile['name']} | Year: {shapefile['year']}")

### Downlaoding shapefiles from IPUMS

With this API key, we can extract geospatial data from the IPUMS API


In [ ]:
from ipumspy.api.extract import NhgisDataset
#Submit extraction data to IPUMS portal
extract = AggregateDataExtract(
    collection="nhgis",
    description="Puerto Rico 2010–2020 vacancy",
    datasets=[
        NhgisDataset(name="2010_SF1a", data_tables=["P12", "H3"], geog_levels=["blck_grp"]),
        NhgisDataset(name="2020_DHCa", data_tables=["P12", "H3"], geog_levels=["blck_grp"]),
    ],
    geographic_extents=["720"]
    #    shapefiles=["720_blck_grp_2020_tl2020"]  to download shapefiles
)


In [ ]:
extract

Getting data from 2010 and 2020 with the variables are Total Housing Units and Occupancy Status as H1 and H3 respectively. 


In [ ]:
#Submit the extract request
ipums.submit_extract(extract)
print(f"Extract ID: {extract.extract_id}")

#Wait for the extract to finish
ipums.wait_for_extract(extract)

In [ ]:
# #Download the extract
current = os.getcwd()
DOWNLOAD_DIR = os.path.join(f"{current}/data/ipums/block") #Change to your storage location

# Create the directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

ipums.download_extract(extract, download_dir=DOWNLOAD_DIR)

In [ ]:
current = os.getcwd()
DOWNLOAD_DIR = os.path.join(f"{current}/data/ipums/block") #Change to your storage location
file_list = os.listdir(DOWNLOAD_DIR)
csv_zip = [f for f in file_list if f.endswith('_csv.zip')]
csv = f"{DOWNLOAD_DIR}/{csv_zip[0]}" 

# Read zip data file in the extract
with ZipFile(csv) as z:
    csv_data = z.namelist()
    print("Contents of zip: ", csv_data)
    
    # Find the correct CSVs using filename patterns
    file_2020 = next(f for f in csv_data if '2020' in f and f.endswith('.csv'))
    file_2010 = next(f for f in csv_data if '2010' in f and f.endswith('.csv'))

    # Read CSVs into DataFrames
    with z.open(file_2020) as f:
        df_2020 = pd.read_csv(f)

    with z.open(file_2010) as f:
        df_2010 = pd.read_csv(f)

In [ ]:
# The NHGIS codes are as follows in the documentation which is downloaded from the IPUMS API 

# Rename columns for dataframe 2020

'''    Table 1:     Sex by Age for Selected Age Categories
    Universe:    Total population
    Source code: P12
    NHGIS code:  U7S
        U7S001:      Total
        U7S002:      Male
        U7S003:      Male: Under 5 years
        U7S004:      Male: 5 to 9 years
        U7S005:      Male: 10 to 14 years
        U7S006:      Male: 15 to 17 years
        U7S007:      Male: 18 and 19 years
        U7S008:      Male: 20 years
        U7S009:      Male: 21 years
        U7S010:      Male: 22 to 24 years
        U7S011:      Male: 25 to 29 years
        U7S012:      Male: 30 to 34 years
        U7S013:      Male: 35 to 39 years
        U7S014:      Male: 40 to 44 years
        U7S015:      Male: 45 to 49 years
        U7S016:      Male: 50 to 54 years
        U7S017:      Male: 55 to 59 years
        U7S018:      Male: 60 and 61 years
        U7S019:      Male: 62 to 64 years
        U7S020:      Male: 65 and 66 years
        U7S021:      Male: 67 to 69 years
        U7S022:      Male: 70 to 74 years
        U7S023:      Male: 75 to 79 years
        U7S024:      Male: 80 to 84 years
        U7S025:      Male: 85 years and over
        U7S026:      Female
        U7S027:      Female: Under 5 years
        U7S028:      Female: 5 to 9 years
        U7S029:      Female: 10 to 14 years
        U7S030:      Female: 15 to 17 years
        U7S031:      Female: 18 and 19 years
        U7S032:      Female: 20 years
        U7S033:      Female: 21 years
        U7S034:      Female: 22 to 24 years
        U7S035:      Female: 25 to 29 years
        U7S036:      Female: 30 to 34 years
        U7S037:      Female: 35 to 39 years
        U7S038:      Female: 40 to 44 years
        U7S039:      Female: 45 to 49 years
        U7S040:      Female: 50 to 54 years
        U7S041:      Female: 55 to 59 years
        U7S042:      Female: 60 and 61 years
        U7S043:      Female: 62 to 64 years
        U7S044:      Female: 65 and 66 years
        U7S045:      Female: 67 to 69 years
        U7S046:      Female: 70 to 74 years
        U7S047:      Female: 75 to 79 years
        U7S048:      Female: 80 to 84 years
        U7S049:      Female: 85 years and over
 
    Table 2:     Occupancy Status
    Universe:    Housing units
    Source code: H3
    NHGIS code:  U9X
        U9X001:      Total
        U9X002:      Occupied
        U9X003:      Vacant
'''


rename_2020 = {
    "U7S001": "Total_Population",
    "U7S002": "Male",
    "U7S003": "Male: Under 5 years",
    "U7S004": "Male: 5 to 9 years",
    "U7S005":      "Male: 10 to 14 years",
    "U7S006":      "Male: 15 to 17 years",
    "U7S007":      "Male: 18 and 19 years",
    "U7S008":      "Male: 20 years",
    "U7S009":      "Male: 21 years",
    "U7S010":      "Male: 22 to 24 years",
    "U7S011":      "Male: 25 to 29 years",
    "U7S012":      "Male: 30 to 34 years",
    "U7S013":      "Male: 35 to 39 years",
    "U7S014":      "Male: 40 to 44 years",
    "U7S015":      "Male: 45 to 49 years",
    "U7S016":      "Male: 50 to 54 years",
    "U7S017":      "Male: 55 to 59 years",
    "U7S018":      "Male: 60 and 61 years",
    "U7S019":      "Male: 62 to 64 years",
    "U7S020":      "Male: 65 and 66 years",
    "U7S021":      "Male: 67 to 69 years",
    "U7S022":      "Male: 70 to 74 years",
    "U7S023":      "Male: 75 to 79 years",
    "U7S024":      "Male: 80 to 84 years",
    "U7S025":      "Male: 85 years and over",
    "U7S026":      "Female",
    "U7S027":      "Female: Under 5 years",
    "U7S028":      "Female: 5 to 9 years",
    "U7S029":      "Female: 10 to 14 years",
    "U7S030":      "Female: 15 to 17 years",
    "U7S031":      "Female: 18 and 19 years",
    "U7S032":      "Female: 20 years",
    "U7S033":      "Female: 21 years",
    "U7S034":      "Female: 22 to 24 years",
    "U7S035":      "Female: 25 to 29 years",
    "U7S036":      "Female: 30 to 34 years",
    "U7S037":      "Female: 35 to 39 years",
    "U7S038":      "Female: 40 to 44 years",
    "U7S039":      "Female: 45 to 49 years",
    "U7S040":      "Female: 50 to 54 years",
    "U7S041":      "Female: 55 to 59 years",
    "U7S042":      "Female: 60 and 61 years",
    "U7S043":      "Female: 62 to 64 years",
    "U7S044":      "Female: 65 and 66 years",
    "U7S045":      "Female: 67 to 69 years",
    "U7S046":      "Female: 70 to 74 years",
    "U7S047":      "Female: 75 to 79 years",
    "U7S048":      "Female: 80 to 84 years",
    "U7S049":      "Female: 85 years and over",
    "U9X001": "Total_Housing_Units",
    "U9X002": "Occupied",
    "U9X003": "Vacant"
}

#Rename columns for dataframe 2010
'''    Table 1:     Housing Units
    Universe:    Housing units
    Source code: H1
    NHGIS code:  IFC
        IFC001:      Total
 
    Table 2:     Occupancy Status
    Universe:    Housing units
    Source code: H3
    NHGIS code:  IFE
        IFE001:      Total
        IFE002:      Occupied
        IFE003:      Vacant'''

rename_2010 = {
    "H76001": "Total_Population",
    "H76002": "Male",
    "H76003": "Male: Under 5 years",
    "H76004": "Male: 5 to 9 years",
    "H76005":      "Male: 10 to 14 years",
    "H76006":      "Male: 15 to 17 years",
    "H76007":      "Male: 18 and 19 years",
    "H76008":      "Male: 20 years",
    "H76009":      "Male: 21 years",
    "H76010":      "Male: 22 to 24 years",
    "H76011":      "Male: 25 to 29 years",
    "H76012":      "Male: 30 to 34 years",
    "H76013":      "Male: 35 to 39 years",
    "H76014":      "Male: 40 to 44 years",
    "H76015":      "Male: 45 to 49 years",
    "H76016":      "Male: 50 to 54 years",
    "H76017":      "Male: 55 to 59 years",
    "H76018":      "Male: 60 and 61 years",
    "H76019":      "Male: 62 to 64 years",
    "H76020":      "Male: 65 and 66 years",
    "H76021":      "Male: 67 to 69 years",
    "H76022":      "Male: 70 to 74 years",
    "H76023":      "Male: 75 to 79 years",
    "H76024":      "Male: 80 to 84 years",
    "H76025":      "Male: 85 years and over",
    "H76026":      "Female",
    "H76027":      "Female: Under 5 years",
    "H76028":      "Female: 5 to 9 years",
    "H76029":      "Female: 10 to 14 years",
    "H76030":      "Female: 15 to 17 years",
    "H76031":      "Female: 18 and 19 years",
    "H76032":      "Female: 20 years",
    "H76033":      "Female: 21 years",
    "H76034":      "Female: 22 to 24 years",
    "H76035":      "Female: 25 to 29 years",
    "H76036":      "Female: 30 to 34 years",
    "H76037":      "Female: 35 to 39 years",
    "H76038":      "Female: 40 to 44 years",
    "H76039":      "Female: 45 to 49 years",
    "H76040":      "Female: 50 to 54 years",
    "H76041":      "Female: 55 to 59 years",
    "H76042":      "Female: 60 and 61 years",
    "H76043":      "Female: 62 to 64 years",
    "H76044":      "Female: 65 and 66 years",
    "H76045":      "Female: 67 to 69 years",
    "H76046":      "Female: 70 to 74 years",
    "H76047":      "Female: 75 to 79 years",
    "H76048":      "Female: 80 to 84 years",
    "H76049":      "Female: 85 years and over",
    "IFC001": "Total_Housing",
    "IFE001": "Total_Housing_Units",
    "IFE002": "Occupied",
    "IFE003": "Vacant"
}

df_2010.rename(columns = rename_2010, inplace = True)
df_2020.rename(columns = rename_2020, inplace = True)

Subsetting Puerto Rico in IPUMS (STATEA 72)

In [ ]:
pr_df_2010 = df_2010[df_2010["STATEA"] == 72]
pr_df_2020 = df_2020[df_2020["STATEA"] == 72]

pr_df_2010 = pr_df_2010.dropna(axis=1, how='all')
pr_df_2020 = pr_df_2020.dropna(axis=1, how='all')

Using our data, we can calculate the population that ire 60 years of age or older

In [ ]:
pop60plus_cols = [
    "Female: 60 and 61 years",
    "Female: 62 to 64 years",
    "Female: 65 and 66 years",
    "Female: 67 to 69 years",
    "Female: 70 to 74 years",
    "Female: 75 to 79 years",
    "Female: 80 to 84 years",
    "Female: 85 years and over",
    "Male: 60 and 61 years",
    "Male: 62 to 64 years",
    "Male: 65 and 66 years",
    "Male: 67 to 69 years",
    "Male: 70 to 74 years",
    "Male: 75 to 79 years",
    "Male: 80 to 84 years",
    "Male: 85 years and over"
]

pr_df_2010["Pop60plus_total"] = pr_df_2010[pop60plus_cols].sum(axis=1)
pr_df_2010["AgedRatio"] = pr_df_2010["Pop60plus_total"] /  pr_df_2010["Total_Population"]
pr_df_2010["VacantRatio"] = pr_df_2010["Vacant"] /  pr_df_2010["Total_Housing_Units"]


pr_df_2020["Pop60plus_total"] = pr_df_2020[pop60plus_cols].sum(axis=1)
pr_df_2020["AgedRatio"] = pr_df_2020["Pop60plus_total"] /  pr_df_2020["Total_Population"]
pr_df_2020["VacantRatio"] = pr_df_2020["Vacant"] /  pr_df_2020["Total_Housing_Units"]

In [ ]:
# Example: plot Female_60plus_total and Male_60plus_total

plt.scatter(pr_df_2010["VacantRatio"], pr_df_2010["AgedRatio"], alpha=0.3)

plt.title("Aged Ratio (60+) over Vacancy Ratio")
plt.xlabel("Vacant Ratio")
plt.ylabel("Aged Ratio") 
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
columns_used = ["Total_Population", "Male", "Female", "Pop60plus_total", "AgedRatio", "Total_Housing_Units", "Occupied", "Vacant", "VacantRatio"]


# Combine first 18 columns with the specific ones, avoiding duplicates
selected_cols = list(pr_df_2020.columns[:24]) + columns_used  

pr_df_2020sel = pr_df_2020[selected_cols]

merged_df = pr_df_2020sel.merge(pr_df_2010[["GISJOIN"]+columns_used], on = "GISJOIN", how = "inner", suffixes=("_2020", "_2010"))
merged_df


merged_df

## Comparing time series 

In [ ]:
merged_df["Total_Pop_Change"] = merged_df["Total_Population_2020"] - merged_df["Total_Population_2010"]
merged_df["Pop60plus_Change"] = merged_df["Pop60plus_total_2020"] - merged_df["Pop60plus_total_2010"]
merged_df["Vacant_Change"] = (merged_df["Vacant_2020"] - merged_df["Vacant_2010"])
merged_df["AgedRatio_Change"].fillna(0, inplace=True) 
merged_df["VacantRatio_Change"] = merged_df["VacantRatio_2020"] - merged_df["VacantRatio_2010"]




merged_df

# Plotting Graphs from IPUMS data:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Example: choose 5 variables from your DataFrame
vars_to_plot = ["Total_Pop_Change", "Pop60plus_Change", "Vacant_Change", "AgedRatio_Change", "VacantRatio_Change"]

# Subset your dataframe
data_subset = merged_df[vars_to_plot]

# Create the pairplot
sns.pairplot(data_subset, diag_kind="hist", corner=True)

plt.suptitle("Pairwise Scatterplots and Histograms", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
merged_df["Vacant_Change"] .hist()

plt.title("Vacancy Ratio Change 2010-2020")

This histogram answers why the Cabo Rojo's Decadal Vacant Change is in the first quantile. There is only one county (frequency) that has less than -2000 value which is belong to Cabo Rojo. Therefore, Cabo Rojo is classified as lower quantile (1st quantile).
 

By looking more further into the histogram and difference bar chart between 2010 and 2020. We can have more confident that quantile classification did not do the right job at interpreation the data.

In [ ]:
# Compute means and standard deviations
means = merged_df[vars_to_plot].mean()
stds = merged_df[vars_to_plot].std()

# Create z-score columns
for var in vars_to_plot:
    z_col = var.replace("_Change", "_dz")
    merged_df[z_col] = (merged_df[var] - means[var]) / stds[var]

In [ ]:
merged_df["Vacant_dz"] .hist()

plt.title("Male/Female Ratio Total Change 2010-2020 (In thousands) Z-score")

In [ ]:
# Select all columns that end with '_z' (z-score columns)
zscore_cols = [col for col in merged_df.columns if col.endswith("_dz")]
data_subset = merged_df[zscore_cols]


# Create the pairplot
sns.pairplot(data_subset, diag_kind="hist", corner=True)

plt.suptitle("Pairwise Scatterplots and Histograms (Z-Scores)", y=1.02)
plt.tight_layout()
plt.show()

This histogram answers why the Cabo Rojo's Decadal Vacant Change is in the first quantile. There is only one county (frequency) that has less than -2000 value which is belong to Cabo Rojo. Therefore, Cabo Rojo is classified as lower quantile (1st quantile).


In [ ]:
merged_df["VacantRatio_quant"] = pd.qcut(merged_df["VacantRatio_zw"], 5, labels = range(1,6)) 


In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter( 
    merged_df["VacantRatio_dz"],
    merged_df["AgedRatio_dz"],
    alpha=0.6,
    color="mediumseagreen",
    edgecolor="black"
)


# Use symmetric log scaling
ax.set_xscale("symlog", linthresh=1)  # linthresh defines the linear range near 0
ax.set_yscale("symlog", linthresh=1)


# Axis formatting
ax.set_xlabel("Vacant Housing Ratio Change")
ax.set_ylabel("Age (60+) Ratio Change")
ax.set_title("Vacant Ratio Change vs Aged Ratio (60+) Change (in Z-Scores)")
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Define the relevant columns
joined_columns = [col for col in pr_sedf_ipums.columns if col.startswith("Male:") or col.startswith("Female:")]

# 2. Ensure Total_Population exists (if not, create it)
if "Total_Population" not in pr_sedf_ipums.columns:
    pr_sedf_ipums["Total_Population"] = pr_sedf_ipums[joined_columns].sum(axis=1)

# 3. Create DataFrame for percentage values
pct_df = pr_sedf_ipums[["Total_Population"] + joined_columns].copy()

# 4. Compute percentage for each age group column
for col in joined_columns:
    pct_df[f"{col}_pct"] = pct_df[col] / pct_df["Total_Population"] * 100

# 5. Drop original count columns, keep only percent columns
pct_df = pct_df[[col for col in pct_df.columns if col.endswith("_pct")]]

# 6. Separate by sex
male_age_percentages = []
female_age_percentages = []

for col in pct_df.columns:
    mean_pct = round(pct_df[col].mean(), 2)
    if col.startswith("Male:"):
        male_age_percentages.append(mean_pct)
    elif col.startswith("Female:"):
        female_age_percentages.append(mean_pct)

# 7. Extract age labels
new_age_columns = [col.split(": ")[1].replace("_pct", "") for col in pct_df.columns if col.startswith("Male:")]

# 8. Create the base DataFrame
population_df = pd.DataFrame({
    "Age": new_age_columns,
    "Male": male_age_percentages,
    "Female": female_age_percentages
})
population = population_df.copy()

# 9. Combine select age groups
# Combine 15-17 and 18-19 into 15-19
population.iloc[3, 0] = "15 to 19 years"
population.iloc[3, 1] += population.iloc[4, 1]
population.iloc[3, 2] += population.iloc[4, 2]
population = population.drop(index=4).reset_index(drop=True)

# Combine 20, 21, 22–24 into 20–24
population.iloc[4, 0] = "20 to 24 years"
population.iloc[4, 1] += population.iloc[5, 1] + population.iloc[6, 1]
population.iloc[4, 2] += population.iloc[5, 2] + population.iloc[6, 2]
population = population.drop(index=[5, 6]).reset_index(drop=True)

# Combine 60–61 and 62–64 into 60–64
population.iloc[12, 0] = "60 to 64 years"
population.iloc[12, 1] += population.iloc[13, 1]
population.iloc[12, 2] += population.iloc[13, 2]
population = population.drop(index=13).reset_index(drop=True)

# Combine 65–66 and 67–69 into 65–69
population.iloc[13, 0] = "65 to 69 years"
population.iloc[13, 1] += population.iloc[14, 1]
population.iloc[13, 2] += population.iloc[14, 2]
population = population.drop(index=14).reset_index(drop=True)

# 10. Prepare for plotting
population["Female_Left"] = 0
population["Female_Width"] = population["Female"]
population["Male_Left"] = -population["Male"]
population["Male_Width"] = population["Male"]

# Round for labels
population["Male"] = population["Male"].round(2)
population["Female"] = population["Female"].round(2)

# 11. Plot the population pyramid
fig = plt.figure(figsize=(15, 10))

plt.barh(y=population["Age"], width=population["Female_Width"], color="#ee7a87", label="Female")
plt.barh(y=population["Age"], width=population["Male_Width"], left=population["Male_Left"],
         color="#4682b4", label="Male")

plt.text(-5, len(population) - 1, "Male", fontsize=25, fontweight="bold")
plt.text(4, len(population) - 1, "Female", fontsize=25, fontweight="bold")

# Add labels
for idx in range(len(population)):
    plt.text(x=population["Male_Left"][idx] - 0.1, y=idx, s="{}%".format(population["Male"][idx]),
             ha="right", va="center", fontsize=15, color="#4682b4")
    plt.text(x=population["Female_Width"][idx] + 0.1, y=idx, s="{}%".format(population["Female"][idx]),
             ha="left", va="center", fontsize=15, color="#ee7a87")

plt.xlim(-7, 7)
plt.xticks(range(-7, 8), ["{}%".format(i) for i in range(-7, 8)])
plt.legend(loc="best")
plt.xlabel("Percent (%)", fontsize=16, fontweight="bold")
plt.ylabel("Age Range", fontsize=16, fontweight="bold")
plt.title("Puerto Rico Mean Age Distribution (Census Blocks)", loc="left", pad=20, fontsize=25, fontweight="bold")

plt.tight_layout()
plt.show()

### Select an individual census block

In [ ]:
# --- STEP 1: Select a census block by GEOID ---
block_id = "721537502022"  # Replace with any valid GEOID
block_df = pr_sedf_ipums[pr_sedf_ipums["GEOID"] == block_id].copy()

assert len(block_df) == 1, f"Expected one row for GEOID {block_id}, but got {len(block_df)}"

# --- STEP 2: Identify male and female age columns ---
male_cols = [col for col in block_df.columns if col.startswith("Male:")]
female_cols = [col for col in block_df.columns if col.startswith("Female:")]
age_labels = [col.split(": ")[1] for col in male_cols]  # Extract age ranges

# --- STEP 3: Get values (choose percent or raw counts) ---
use_percent = True  # Set to False if you want raw counts

if use_percent:
    total_pop = block_df[male_cols + female_cols].sum(axis=1).values[0]
    male_vals = (block_df[male_cols].values[0] / total_pop) * 100
    female_vals = (block_df[female_cols].values[0] / total_pop) * 100
    value_label = "Percent (%)"
else:
    male_vals = block_df[male_cols].values[0]
    female_vals = block_df[female_cols].values[0]
    value_label = "Population Count"

# --- STEP 4: Build plotting dataframe ---
population = pd.DataFrame({
    "Age": age_labels,
    "Male": male_vals,
    "Female": female_vals
})

population["Female_Left"] = 0
population["Female_Width"] = population["Female"]
population["Male_Left"] = -population["Male"]
population["Male_Width"] = population["Male"]

# --- STEP 5: Plot the pyramid ---
fig = plt.figure(figsize=(12, 8))

plt.barh(y=population["Age"], width=population["Female_Width"], color="#ee7a87", label="Female")
plt.barh(y=population["Age"], width=population["Male_Width"], left=population["Male_Left"],
         color="#4682b4", label="Male")

plt.text(-5, len(population) - 1, "Male", fontsize=20, fontweight="bold")
plt.text(4, len(population) - 1, "Female", fontsize=20, fontweight="bold")

for idx in range(len(population)):
    plt.text(x=population["Male_Left"][idx] - 0.1, y=idx, s=f"{round(population['Male'][idx], 1)}",
             ha="right", va="center", fontsize=12, color="#4682b4")
    plt.text(x=population["Female_Width"][idx] + 0.1, y=idx, s=f"{round(population['Female'][idx], 1)}",
             ha="left", va="center", fontsize=12, color="#ee7a87")

xlim_val = max(population[["Male", "Female"]].max()) * 1.2
plt.xlim(-xlim_val, xlim_val)
plt.xticks([])
plt.legend(loc="best")

plt.xlabel(value_label, fontsize=14)
plt.ylabel("Age Range", fontsize=14)
plt.title(f"Census Block {block_id} Age Distribution", fontsize=18, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
bg_name = "Block Group 3"

df_gb = pr_sedf_ipums[pr_sedf_ipums["NAME_x"] == bg_name ]


# 1. Define the relevant columns
joined_columns = [col for col in df_gb.columns if col.startswith("Male:") or col.startswith("Female:")]

# 2. Ensure Total_Population exists (if not, create it)
if "Total_Population" not in df_gb.columns:
    df_gb["Total_Population"] = df_gb[joined_columns].sum(axis=1)

# 3. Create DataFrame for percentage values
pct_df = df_gb[["Total_Population"] + joined_columns].copy()

# 4. Compute percentage for each age group column
for col in joined_columns:
    pct_df[f"{col}_pct"] = pct_df[col] / pct_df["Total_Population"] * 100

# 5. Drop original count columns, keep only percent columns
pct_df = pct_df[[col for col in pct_df.columns if col.endswith("_pct")]]

# 6. Separate by sex
male_age_percentages = []
female_age_percentages = []

for col in pct_df.columns:
    mean_pct = round(pct_df[col].mean(), 2)
    if col.startswith("Male:"):
        male_age_percentages.append(mean_pct)
    elif col.startswith("Female:"):
        female_age_percentages.append(mean_pct)

# 7. Extract age labels
new_age_columns = [col.split(": ")[1].replace("_pct", "") for col in pct_df.columns if col.startswith("Male:")]

# 8. Create the base DataFrame
population_df = pd.DataFrame({
    "Age": new_age_columns,
    "Male": male_age_percentages,
    "Female": female_age_percentages
})
population = population_df.copy()

# 9. Combine select age groups
# Combine 15-17 and 18-19 into 15-19
population.iloc[3, 0] = "15 to 19 years"
population.iloc[3, 1] += population.iloc[4, 1]
population.iloc[3, 2] += population.iloc[4, 2]
population = population.drop(index=4).reset_index(drop=True)

# Combine 20, 21, 22–24 into 20–24
population.iloc[4, 0] = "20 to 24 years"
population.iloc[4, 1] += population.iloc[5, 1] + population.iloc[6, 1]
population.iloc[4, 2] += population.iloc[5, 2] + population.iloc[6, 2]
population = population.drop(index=[5, 6]).reset_index(drop=True)

# Combine 60–61 and 62–64 into 60–64
population.iloc[12, 0] = "60 to 64 years"
population.iloc[12, 1] += population.iloc[13, 1]
population.iloc[12, 2] += population.iloc[13, 2]
population = population.drop(index=13).reset_index(drop=True)

# Combine 65–66 and 67–69 into 65–69
population.iloc[13, 0] = "65 to 69 years"
population.iloc[13, 1] += population.iloc[14, 1]
population.iloc[13, 2] += population.iloc[14, 2]
population = population.drop(index=14).reset_index(drop=True)

# 10. Prepare for plotting
population["Female_Left"] = 0
population["Female_Width"] = population["Female"]
population["Male_Left"] = -population["Male"]
population["Male_Width"] = population["Male"]

# Round for labels
population["Male"] = population["Male"].round(2)
population["Female"] = population["Female"].round(2)

# 11. Plot the population pyramid
fig = plt.figure(figsize=(15, 10))

plt.barh(y=population["Age"], width=population["Female_Width"], color="#ee7a87", label="Female")
plt.barh(y=population["Age"], width=population["Male_Width"], left=population["Male_Left"],
         color="#4682b4", label="Male")

plt.text(-5, len(population) - 1, "Male", fontsize=25, fontweight="bold")
plt.text(4, len(population) - 1, "Female", fontsize=25, fontweight="bold")

# Add labels
for idx in range(len(population)):
    plt.text(x=population["Male_Left"][idx] - 0.1, y=idx, s="{}%".format(population["Male"][idx]),
             ha="right", va="center", fontsize=15, color="#4682b4")
    plt.text(x=population["Female_Width"][idx] + 0.1, y=idx, s="{}%".format(population["Female"][idx]),
             ha="left", va="center", fontsize=15, color="#ee7a87")

plt.xlim(-7, 7)
plt.xticks(range(-7, 8), ["{}%".format(i) for i in range(-7, 8)])
plt.legend(loc="best")
plt.xlabel("Percent (%)", fontsize=16, fontweight="bold")
plt.ylabel("Age Range", fontsize=16, fontweight="bold")
plt.title(f"Puerto Rico Mean Age Distribution ({bg_name})", loc="left", pad=20, fontsize=25, fontweight="bold")

plt.tight_layout()
plt.show()

## Summarizing and Plotting DAta

Let's combine the Block Groups into Counties and Map the results.



## HIEU SUMMARIZE merged_df TO COUNTY (pd_county_df) using "sum" to add:  "Total_Population", "Male", "Female", "Pop60plus_total",  "Total_Housing_Units", "Occupied", "Vacant",  for all have _2010 and _2020 at the end. 



### Hieu Recalculate CHange

In [ ]:
pr_county_df["AgedRatio_2010"] = pr_county_df["Pop60plus_total_2010"] /  pr_county_df["Total_Population_2010"]
pr_county_df["VacantRatio_2010"] = pr_county_df["Vacant_2010"] /  pr_county_df["Total_Housing_Units_2010"]


pr_county_df["AgedRatio_2020"] = pr_county_df["Pop60plus_total_2020"] /  pr_county_df["Total_Population_2020"]
pr_county_df["VacantRatio_2020"] = pr_county_df["Vacant_2020"] /  pr_county_df["Total_Housing_Units_2020"]

In [ ]:
pr_county_df["Total_Pop_Change"] = pr_county_df["Total_Population_2020"] - pr_county_df["Total_Population_2010"]
pr_county_df["Pop60plus_Change"] = pr_county_df["Pop60plus_total_2020"] - pr_county_df["Pop60plus_total_2010"]
pr_county_df["Vacant_Change"] = (pr_county_df["Vacant_2020"] - pr_county_df["Vacant_2010"])

pr_county_df["VacantRatio_Change"] = pr_county_df["VacantRatio_2020"] - pr_county_df["VacantRatio_2010"]

pr_county_df["AgedRatio_Change"] = (pr_county_df["AgedRatio_2020"] - pr_county_df["AgedRatio_2010"])


pr_county_df

In [ ]:
# Compute means and standard deviations
means = pr_county_df[vars_to_plot].mean()
stds = pr_county_df[vars_to_plot].std()

# Create z-score columns
for var in vars_to_plot:
    z_col = var.replace("_Change", "_dz")
    pr_county_df[z_col] = (pr_county_df[var] - means[var]) / stds[var]

##

In [ ]:
# Filter for lowest and highest vacancy quantile groups
lowest = pr_county_df[pr_county_df["AgedRatio_dz"] == 1]
highest = pr_county_df[pr_county_df["AgedRatio_dz"] == 5]


# Use COUNTYA for grouping (could be COUNTY if preferred)
# Step 1: Find common counties between the two groups
common_counties = sorted(
    set(lowest["COUNTY"]).intersection(set(highest["COUNTY"]))
)

# Step 2: Group and sum vacant housing units for 2010 and 2020, reindexed by common counties
vacant_low_2010 = lowest[lowest["COUNTY"].isin(common_counties)] \
    .groupby("COUNTY")["VacantRatio_2010"].sum().reindex(common_counties)
vacant_low_2020 = lowest[lowest["COUNTYA"].isin(common_counties)] \
    .groupby("COUNTY")["VacantRatio_2020"].sum().reindex(common_counties)

vacant_high_2010 = highest[highest["COUNTY"].isin(common_counties)] \
    .groupby("COUNTY")["VacantRatio_2010"].sum().reindex(common_counties)
vacant_high_2020 = highest[highest["COUNTY"].isin(common_counties)] \
    .groupby("COUNTY")["VacantRatio_2020"].sum().reindex(common_counties)

# Step 3: Plot
x = np.arange(len(common_counties))  # consistent X positions

fig, ax = plt.subplots(figsize=(12, 6))

ax.scatter(x, vacant_low_2010.values, label='Lowest Age Ratio 2010', color='skyblue', marker='o', s=60)
ax.scatter(x, vacant_low_2020.values, label='Lowest Age Ratio 2020', color='blue', marker='o', s=60)
ax.scatter(x, vacant_high_2010.values, label='Highest Age Ratio 2010', color='orange', marker='^', s=60)
ax.scatter(x, vacant_high_2020.values, label='Highest Age Ratio 2020', color='darkred', marker='^', s=60)

# Formatting
ax.set_ylabel('Vacant Housing Ratio')
ax.set_xlabel('County (COUNTYA)')
ax.set_title('Vacant Housing Units by County: Lowest vs Highest Age Ratio Change Quantile (2010 & 2020)')
ax.set_xticks(x)
ax.set_xticklabels(common_counties, rotation=90)
ax.legend()
ax.grid(True, axis='y')

plt.tight_layout()
plt.show()


# Mapping data



## Start preprocessing the Census data downloaded from IPUMS API

To map the shapefile to the map, we have to publish the shapefile as feature layer on MapViewer on ArcGIS Online by adding layer from your shapefile zipped folder. 

This lesson is public facing so we already published the layers for the learners.


In [ ]:
# Explicit interactive authentication
gis = GIS("https://www.arcgis.com", authenticate='interactive')


# Get the layer from the published data

results = gis.content.search("Census County Esri_US_Federal_Data", item_type="Feature Layer")
for r in results:
    print(r)

Select the position of the "Cansus Zip Code Tabulation Areas" layer starting with position the first position as `0`. 

In [ ]:
#select the appropiate item
zip_item = results[3]
item_id = zip_item.id
print(f"Title: {zip_item.title}")
print(f"ID: {item_id}")


With the item's ID (ccfe5a9e3eea4ec68b7945b610422b0a) from ablove, we can  use the layers for analysis

In [ ]:
from arcgis.features import FeatureLayer
# Authenticate interactively
gis = GIS("https://www.arcgis.com", authenticate='interactive')
layer = gis.content.get(item_id).layers[0] # The layer of PR county 2020 published to the portal
# Print all field names
field_names = [field['name'] for field in layer.properties.fields]
print(field_names)
print(layer.url)
layer = FeatureLayer(layer.url)

## Hieu Change this for Change this for County to display the "COUNTY" name in the label and either VacancyRatio_dz 

In [ ]:
# Build unique value info entries
unique_value_infos = []
for i in range(5):
    unique_value_infos.append({
        "value": i + 1,
        "label": quantile_labels[i],
        "symbol": symbols[i]
    })

# Define the unique value renderer
pr_uvr = {
    "type": "uniqueValue",
    "field1": "Total_Population",
    "uniqueValueInfos": unique_value_infos
}

# Define labeling info
pr_labeling_info = [
    {
        "labelExpression": "[NAME_x]",
        "labelPlacement": "esriServerPolygonPlacementAlwaysHorizontal",
        "repeatLabel": True,
        "symbol": {
            "type": "esriTS",
            "color": [0, 0, 0, 255],
            "font": {
                "family": "Arial",
                "size": 12
            },
            "horizontalAlignment": "center",
            "kerning": True
        }
    }
]

# Layer display options
pr_options_dict = {
    "showLabels": True,
    "layerDefinition": {
        "drawingInfo": {
            "labelingInfo": pr_labeling_info,
            "renderer": pr_uvr
        }
    },
    "opacity": 0.5,
    "title": "Vacant Housing Units Decadal Change"
}



gis = GIS()
m1 = gis.map("Puerto Rico") 
 
m1.content.add(pr_sedf_ipums, options = pr_options_dict) 
# update the legend label for LECZ layer in Notebooks
 
m1.legend.enabled = True

m1

# Search for LECZ MERIT-DEM layer on ArcGIS Online's Living Atlas portal.

With the arcgis package, we can search online for the polygons of the LECZ areas in Puerto Rico


## Hieu  Overlay LECZ polygons and add a classification column of 1 or 0 to the counties dataframe

## Plot bloxplot showing differences in Vacant and Aged Ratios in the two groups (in lecz, not in lecz)

## Display map with counties in LEZ only with colors displaying AgedRatio_dz
